In [1]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet,Transformer
from easydict import EasyDict

In [2]:
%load_ext autoreload
%autoreload 2

#### Load config file and print hyperparameters

In [3]:
model_dict = {
    'ynet': YNet,
    'transformer': Transformer,
}
CONFIG_FILE_PATH = 'config/sdd_trajnet.yaml'  # yaml config file containing all the hyperparameters
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
params = EasyDict(params)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params



{'resize': 0.25,
 'batch_size': 4,
 'viz_epoch': 10,
 'learning_rate': 0.0001,
 'num_epochs': 300,
 'waypoints': [11],
 'temperature': 1.0,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'use_features_only': False,
 'unfreeze': 150,
 'use_TTST': True,
 'rel_threshold': 0.01,
 'use_CWS': False,
 'CWS_params': 'None',
 'dataset_name': 'sdd',
 'train_data_path': '../dataset/SDD/train_trajnet.pkl',
 'train_image_path': '../dataset/SDD/train',
 'val_data_path': '../dataset/SDD/test_trajnet.pkl',
 'val_image_path': '../dataset/SDD/test',
 'num_goals': 20,
 'num_traj': 1,
 'experiment_name': 'sdd_trajnet',
 'model': {'name': 'ynet',
  'kwargs': {'obs_len': 8,
   'pred_len': 12,
   'segmentation_model_fp': 'segmentation_models/inD_segmentation.pth',
   'use_features_only': False,
   'semantic_classes': 6,
   'encoder_channels': [32, 32, 64, 64, 64],
   'decoder_channels': [64, 64, 64, 32, 32],
   '

#### Some hyperparameters and settings

In [4]:
EXPERIMENT_NAME = params.experiment_name  # arbitrary name for this experiment
DATASET_NAME = params.dataset_name

TRAIN_DATA_PATH = params.train_data_path
TRAIN_IMAGE_PATH = params.train_image_path
VAL_DATA_PATH = params.val_data_path
VAL_IMAGE_PATH = params.val_image_path
# OBS_LEN = params.obs_length  # in timesteps
# PRED_LEN = params.pred_len  # in timesteps
NUM_GOALS = params.num_goals  # K_e
NUM_TRAJ = params.num_traj  # K_a

BATCH_SIZE = params.batch_size


#### Load preprocessed Data

In [5]:
df_train = pd.read_pickle(TRAIN_DATA_PATH)
df_val = pd.read_pickle(VAL_DATA_PATH)

In [6]:
df_train.head()

,frame,trackId,x,y,sceneId,metaId
0,0,100,757.0,918.5,bookstore_0,0
1,12,100,765.0,918.5,bookstore_0,0
2,24,100,773.0,918.5,bookstore_0,0
3,36,100,780.5,918.5,bookstore_0,0
4,48,100,788.5,919.5,bookstore_0,0


#### Initiate model

In [7]:
model = model_dict[params.model.name](**params.model.kwargs)

/home/nvidia-3080/anaconda3/envs/hxy/lib/python3.9/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'segmentation_models_pytorch.unet.model.Unet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/nvidia-3080/anaconda3/envs/hxy/lib/python3.9/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/nvidia-3080/anaconda3/envs/hxy/lib/python3.9/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [8]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH, 
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|█████████████████████| 240/240 [00:00<00:00, 1180.33it/s]
/media/nvidia-3080/35aae167-c57e-42a4-aae3-7dbbf1275d98/hxy/TrajectoryPredict/CodeBase/utils/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(trajectories), meta, scene_list
Prepare Dataset: 100%|███████████████████████| 17/17 [00:00<00:00, 1408.43it/s]


Start training


Epoch:   0%|                                           | 0/300 [00:00<?, ?it/s]/media/nvidia-3080/35aae167-c57e-42a4-aae3-7dbbf1275d98/hxy/TrajectoryPredict/CodeBase/utils/dataloader.py:46: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/torch/csrc/utils/tensor_new.cpp:201.)
  return torch.Tensor(trajectories).squeeze(0), meta, scene[0]


traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])


/home/nvidia-3080/anaconda3/envs/hxy/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([3, 8, 2]) gt:torch.Size([3, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 1

Epoch:   0%|                                           | 0/300 [00:05<?, ?it/s]

traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])
traj:torch.Size([4, 8, 2]) gt:torch.Size([4, 12, 2])


KeyboardInterrupt: 